In [1]:
import numpy as np
from torch import nn, optim
import torch
import random
import time

🤔思考：如何加载数据？

In [2]:
with open("labels.txt", "r") as f:
    raw = f.readlines()

tags = []
data = []
for l in raw:
    tags.append(int(l[0]))  # 每行的第一个字符是标签
    d = l[1:-1]  # 去掉标签和换行符
    d = map(float, tuple(d))  # 将字符串转换为tuple，数字转换为float，方便后续转为tensor
    # tuple相对于list更省内存，因为tuple是不可变的，对象所含method更少
    data.append(tuple(d))

# 将标签和数据转为tensor，方便后续切分训练集和测试集
data = torch.tensor(data)
tags = torch.tensor(tags)

# 划分训练集和测试集
train_test_ratio = 0.8
train_size = int(train_test_ratio * len(data))
test_size = len(data) - train_size
data_train = data[:train_size]
data_test = data[train_size:]
tags_train = tags[:train_size]
tags_test = tags[train_size:]

In [3]:
# 直接套用d2l网站上的代码，没有改动
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]


batch_size = 10

w = torch.normal(0, 0.01, size=(
    len(data[0]), 1), requires_grad=True, dtype=torch.float32)  # 对每个像素都有一个权重
b = torch.zeros(1, requires_grad=True)

👇方案2：softmax回归

数学原理：

Softmax回归是一种分类算法，它可以将输入数据映射到一个固定维度的输出向量，每个元素对应于输入数据属于各个类别的概率。

- Softmax回归的公式：

$$
\begin{align*}
\hat{y} &= \text{softmax}(Wx+b) \\

\text{softmax}(x_i) &= \frac{\exp(x_i)}{\sum_j \exp(x_j)}
\end{align*}
$$


其中，$W$和$b$是模型的参数，$x$是输入数据，$\hat{y}$是模型的输出。

- Softmax回归的损失函数：

$$
\begin{align*}
L &= -\frac{1}{N}\sum_{i=1}^N\sum_{j=1}^K t_{ij}\log\hat{y}_{ij} \\
&= -\frac{1}{N}\sum_{i=1}^N\sum_{j=1}^K t_{ij}\log\frac{\exp(x_i^T w_j + b_j)}{\sum_{k=1}^K \exp(x_i^T w_k + b_k)}
\end{align*}
$$

其中，$t_{ij}$是样本$i$的真实标签，$K$是类别数。

- Softmax回归的优化算法：

$$
\begin{align*}
\text{repeat until convergence} \\
\text{for each example} \\
&\text{compute } \hat{y}_i = \text{softmax}(Wx_i+b) \\
&\text{compute } \delta_i = \hat{y}_i - t_i \\
&\text{compute } \nabla_w L = \frac{1}{N}\sum_{i=1}^N \delta_i x_i^T \\
&\text{compute } \nabla_b L = \frac{1}{N}\sum_{i=1}^N \delta_i \\
&\text{update } w = w - \eta \nabla_w L \\
&\text{update } b = b - \eta \nabla_b L
\end{align*}
$$

- [Softmax的实现：](https://zh.d2l.ai/chapter_linear-networks/softmax-regression-concise.html#subsec-softmax-implementation-revisited)

>  尽管我们要计算指数函数，但我们最终在计算交叉熵损失时会取它们的对数。 通过将softmax和交叉熵结合在一
>  起，可以避免反向传播过程中可能会困扰我们的数值稳定性问题。 如下面的等式所示，我们
>避免计算 $\exp(o_j  - \max(o_k))$， 而可以直接使用$o_j - \max(o_k)$，因为$\log(\exp(\cdot))$被抵消了。

>我们也希望保留传统的softmax函数，以备我们需要评估通过模型输出的概率。 但是，我们没有将softmax概率传
>递到损失函数中， 而是在交叉熵损失函数中传递未规范化的预测，并同时计算softmax及其对数， 这是一种类似
>“LogSumExp技巧”的聪明方式。

In [4]:
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)


def accuracy(y_hat, y):
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:  # 多分类
        y_hat = y_hat.argmax(axis=1)    # 取最大值作为预测类别
    cmp = y_hat.type(y.dtype) == y  # 预测正确的位置为True
    return float(cmp.type(y.dtype).sum()/len(y))


# 定义数据集
batch_size = 32
net = nn.Sequential(
    nn.Linear(384, 10)  # 定义线性层，输入384，输出10个类别
)
net.apply(init_weights)  # 初始化权重
loss = nn.CrossEntropyLoss(reduction='none')  # “LogSumExp技巧”
trainer = torch.optim.SGD(net.parameters(), lr=0.015)  # 定义优化器和学习率
num_epochs = 32  # 训练轮数


for epoch in range(num_epochs):
    net.train()  # 切换到训练模式
    for X, y in data_iter(batch_size, data_train, tags_train):
        # 计算梯度并更新参数
        y_hat = net(X)
        l = loss(y_hat, y)

        # 使用PyTorch内置的优化器和损失函数
        trainer.zero_grad()
        l.mean().backward() 
        trainer.step()  # 更新参数

    # 计算训练准确度
    net.eval()  # 切换到测试模式
    with torch.no_grad():  # 虽然net.eval()可以切换模型为评估模式，但在评估过程中，还建议结合torch.no_grad()上下文管理器来进一步优化性能。
        acc = accuracy(net(data_train), tags_train)
    print(f'epoch {epoch+1:2d}, loss: {l.mean().item():.3f}, acc: {acc:.3f}')
    # net.eval()
    # acc = accuracy(net(data_train), tags_train)
    # print(f'epoch {epoch+1:2d}, loss: {l.mean().item():.3f}, acc: {acc:.3f}')

epoch  1, loss: 1.614, acc: 0.894
epoch  2, loss: 1.000, acc: 0.974
epoch  3, loss: 0.848, acc: 0.974
epoch  4, loss: 0.632, acc: 0.974
epoch  5, loss: 0.413, acc: 0.974
epoch  6, loss: 0.327, acc: 0.974
epoch  7, loss: 0.426, acc: 0.974
epoch  8, loss: 0.263, acc: 0.974
epoch  9, loss: 0.389, acc: 0.974
epoch 10, loss: 0.201, acc: 0.974
epoch 11, loss: 0.177, acc: 0.974
epoch 12, loss: 0.304, acc: 0.974
epoch 13, loss: 0.153, acc: 0.974
epoch 14, loss: 0.126, acc: 0.974
epoch 15, loss: 0.286, acc: 0.974
epoch 16, loss: 0.260, acc: 0.974
epoch 17, loss: 0.260, acc: 0.974
epoch 18, loss: 0.278, acc: 0.974
epoch 19, loss: 0.102, acc: 0.974
epoch 20, loss: 0.093, acc: 0.974
epoch 21, loss: 0.263, acc: 0.974
epoch 22, loss: 0.227, acc: 0.974
epoch 23, loss: 0.419, acc: 0.974
epoch 24, loss: 0.205, acc: 0.974
epoch 25, loss: 0.210, acc: 0.974
epoch 26, loss: 0.393, acc: 0.974
epoch 27, loss: 0.090, acc: 0.974
epoch 28, loss: 0.332, acc: 0.974
epoch 29, loss: 0.665, acc: 0.974
epoch 30, loss

In [5]:
with torch.no_grad():
    y_hat = net(data_test)
    y = tags_test  # 标签转为LongTensor类型
    acc = accuracy(y_hat, y)
    print(f'loss: {l.mean().item():.3f}, acc: {acc:.3f}')

loss: 0.251, acc: 0.965
